In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
fires = pd.read_csv("fires.csv")

fires["date"] = pd.to_datetime(fires["Дата составления"])
del fires["Дата составления"]
fires['start_date'] = pd.to_datetime(fires['Дата начала'])
del fires['Дата начала']
fires['end_date'] = pd.to_datetime(fires['Дата оконч.'])
del fires['Дата оконч.']
fires['Нач.форм.штабеля'] = pd.to_datetime(fires['Нач.форм.штабеля'])

supplies = pd.read_csv("supplies.csv")
supplies["start_date"] = pd.to_datetime(supplies["ВыгрузкаНаСклад"])
del supplies["ВыгрузкаНаСклад"]
supplies["end_date"] = pd.to_datetime(supplies["ПогрузкаНаСудно"])
del supplies["ПогрузкаНаСудно"]
# merged = pd.merge_asof(
#     fires.sort_values("date"),
#     suppliers.sort_values("start_date"),
#     left_on="date",
#     right_on="start_date",
#     by="supplier_id",
#     direction="backward"
# )

weather = pd.read_csv("weather.csv")
weather["date"] = pd.to_datetime(weather["date"])


In [4]:
# Группировка supplies по номеру склада и штабеля с основными агрегатами
grouped_supplies = supplies.groupby(['Склад', 'Штабель']).agg(
    entries=('Наим. ЕТСНГ', 'count'),
    total_on_stock=('На склад, тн', 'sum'),
    total_on_ship=('На судно, тн', 'sum'),
    start_min=('start_date', 'min'),
    start_max=('start_date', 'max'),
    end_min=('end_date', 'min'),
    end_max=('end_date', 'max')
).reset_index()
print("Группировка supplies по номеру склада и штабеля выполнена.")
# сортировка и вывод первых строк
grouped_supplies = grouped_supplies.sort_values(['Склад', 'Штабель']).reset_index(drop=True)
grouped_supplies.head()

Группировка supplies по номеру склада и штабеля выполнена.


,Склад,Штабель,entries,total_on_stock,total_on_ship,start_min,start_max,end_min,end_max
0,3,1,32,4.042382e+05,4.038484e+05,2019-01-02,2019-08-19,2019-02-08,2019-09-10
1,3,2,42,2.974619e+05,2.972633e+05,2019-06-12,2020-01-20,2019-07-11,2020-02-11
2,3,4,183,1.844732e+06,1.840972e+06,2019-03-31,2020-03-20,2019-04-12,2020-04-18
3,3,5,107,8.072703e+05,8.057269e+05,2019-01-24,2020-05-17,2019-02-18,2020-06-23
4,3,6,37,2.772846e+05,2.772846e+05,2020-07-28,2020-09-28,2020-10-13,2020-10-13


In [13]:
res_supplies = pd.DataFrame()
for i in range(len(grouped_supplies)):
    group_row = grouped_supplies.loc[i]
    data = supplies[(supplies['Склад'] == group_row['Склад']) & (supplies['Штабель'] == group_row['Штабель'])]
    min_start_date = group_row['start_min']
    max_end_date = group_row['end_max']
    # пропускаем группы без корректных дат
    if pd.isna(min_start_date) or pd.isna(max_end_date):
        continue
    dates_range = pd.date_range(start=min_start_date, end=max_end_date, freq='D')
    idx = pd.to_datetime(dates_range).normalize()
    for type_col in data['Наим. ЕТСНГ'].unique():
        # создаём локальную таблицу с одной строкой на каждую дату диапазона
        local_df = pd.DataFrame({'date': dates_range})
        local_df['Склад'] = group_row['Склад']
        local_df['Штабель'] = group_row['Штабель']
        local_df['Наим. ЕТСНГ'] = type_col
        # рассчитываем количество угля на складе в каждую дату: кумулятивный приход - кумулятивная отгрузка
        temp = data[data['Наим. ЕТСНГ'] == type_col].copy()
        if temp.empty:
            local_df['Масса угля'] = 0
        else:
            # группируем приход и отгрузку по датам (нормализованные даты)
            arrivals = temp.groupby(temp['start_date'].dt.normalize())['На склад, тн'].sum()
            shipments = temp.groupby(temp['end_date'].dt.normalize())['На судно, тн'].sum()
            # создаём индекс по всем датам диапазона и получаем накопленные суммы
            arrivals_cum = arrivals.reindex(idx, fill_value=0).cumsum()
            shipments_cum = shipments.reindex(idx, fill_value=0).cumsum()
            stock_on_date = (arrivals_cum - shipments_cum).clip(lower=0)
            local_df['Масса угля'] = local_df['date'].dt.normalize().map(stock_on_date).fillna(0).astype(float)
        res_supplies = pd.concat([res_supplies, local_df], ignore_index=True)
# приводим типы и сортируем
res_supplies['date'] = pd.to_datetime(res_supplies['date'])
res_supplies = res_supplies.sort_values(['Склад', 'Штабель', 'date']).reset_index(drop=True)
# Добавим погодные данные из таблицы weather (агрегируем по дате)
# нормализуем даты в обеих таблицах и агрегируем weather: числовые - среднее, нечисловые - первый
res_supplies['date_norm'] = res_supplies['date'].dt.normalize()
weather_norm = weather.copy()
weather_norm['date_norm'] = pd.to_datetime(weather_norm['date']).dt.normalize()
numeric_cols = weather_norm.select_dtypes(include=[np.number]).columns.tolist()
non_numeric = [c for c in weather_norm.columns if c not in numeric_cols + ['date', 'date_norm']]
agg_dict = {c: 'mean' for c in numeric_cols}
for c in non_numeric:
    agg_dict[c] = 'first'
if len(agg_dict) > 0:
    weather_by_date = weather_norm.groupby('date_norm').agg(agg_dict).reset_index()
else:
    # защитный вариант: если в weather нет колонок, оставляем пустой фрейм
    weather_by_date = weather_norm[['date_norm']].drop_duplicates().reset_index(drop=True)
# объединяем по нормализованной дате
res_supplies = res_supplies.merge(weather_by_date, left_on='date_norm', right_on='date_norm', how='left')
# Добавим информацию о пожарах из таблицы fires
# Подготовим интервалы пожаров и дату начала формирования штабеля для каждой группы
fires_copy = fires.copy()
# убедимся, что даты в нужном формате
for dt_col in ['start_date', 'end_date', 'Нач.форм.штабеля']:
    if dt_col in fires_copy.columns:
        fires_copy[dt_col] = pd.to_datetime(fires_copy[dt_col], errors='coerce')
# выберем ключи сопоставления (если в fires есть 'Наим. ЕТСНГ', будем сопоставлять по нему тоже)
match_keys = ['Склад', 'Штабель']
if 'Наим. ЕТСНГ' in fires_copy.columns and 'Наим. ЕТСНГ' in res_supplies.columns:
    match_keys.append('Наим. ЕТСНГ')
# создадим словари: интервалы, списки стартов, и минимальная дата начала формирования
fires_intervals = {}
fires_starts = {}
fires_form_start = {}
for _, r in fires_copy.iterrows():
    try:
        key = tuple(r[k] for k in match_keys)
    except Exception:
        continue
    s = r.get('start_date')
    e = r.get('end_date')
    if pd.isna(s):
        continue
    s_n = pd.to_datetime(s).normalize()
    e_n = pd.to_datetime(e).normalize() if (not pd.isna(e)) else s_n
    fires_intervals.setdefault(key, []).append((s_n, e_n))
    fires_starts.setdefault(key, []).append(s_n)
    form = r.get('Нач.форм.штабеля') if 'Нач.форм.штабеля' in r else None
    if not pd.isna(form):
        cur = fires_form_start.get(key)
        if cur is None or pd.to_datetime(form) < cur:
            fires_form_start[key] = pd.to_datetime(form)
# функция для расчёта дней до пожара и флага пожара сейчас
def days_until_fire(row):
    try:
        key = tuple(row[k] for k in match_keys)
    except Exception:
        return 10000
    date_norm = pd.to_datetime(row['date']).normalize()
    intervals = fires_intervals.get(key, [])
    # если пожар идёт сейчас (входит в любой интервал) -> 0
    for (s,e) in intervals:
        if s <= date_norm <= e:
            return 0
    # иначе ищем ближайшую дату старта пожара после текущей даты
    starts = sorted(set(fires_starts.get(key, [])))
    for st in starts:
        if st > date_norm:
            return int((st - date_norm).days)
    return 10000
# применим функцию к res_supplies
res_supplies['До пожара'] = res_supplies.apply(days_until_fire, axis=1)
# добавим начало формирования штабеля (если известно для группы)
def get_stack_form_start(row):
    try:
        key = tuple(row[k] for k in match_keys)
    except Exception:
        return pd.NaT
    return fires_form_start.get(key, pd.NaT)
res_supplies['Начало формирования штабеля'] = res_supplies.apply(get_stack_form_start, axis=1)
# удаляем служебный нормализованный столбец, если остался
if 'date_norm' in res_supplies.columns:
    res_supplies.drop(columns=['date_norm'], inplace=True)
res_supplies.head()

,date,Склад,Штабель,Наим. ЕТСНГ,Масса угля,t,p,humidity,precipitation,wind_dir,v_avg,v_max,cloudcover,visibility,weather_code,До пожара,Начало формирования штабеля
0,2019-01-02,3,1,E5,11984.1925,7.529167,1013.716667,75.541667,0.216667,187.625000,18.733333,27.975000,86.791667,NaN,35.208333,214,2019-01-01
1,2019-01-02,3,1,B2,0.0000,7.529167,1013.716667,75.541667,0.216667,187.625000,18.733333,27.975000,86.791667,NaN,35.208333,214,2019-01-01
2,2019-01-02,3,1,A1,0.0000,7.529167,1013.716667,75.541667,0.216667,187.625000,18.733333,27.975000,86.791667,NaN,35.208333,214,2019-01-01
3,2019-01-03,3,1,E5,11984.1925,5.608333,1011.741667,87.708333,0.358333,172.666667,15.995833,23.920833,98.666667,NaN,41.833333,213,2019-01-01
4,2019-01-03,3,1,B2,0.0000,5.608333,1011.741667,87.708333,0.358333,172.666667,15.995833,23.920833,98.666667,NaN,41.833333,213,2019-01-01


In [14]:
res_supplies.to_csv("prepared_supplies.csv", index=False)